# Install Package

In [ ]:
# install pybaseball (for Google Colab)
!pip install pybaseball

# Import Packages

In [2]:
import numpy as np
import pandas as pd
import math
from pybaseball import statcast

# Extracting Statcast data (by using pybaseball)

In [ ]:
# Specify date(e.g. from 2024-05-01 to 2024-05-10)
pyb_df = statcast('2024-05-01', '2024-05-10')

# Limit to regular season data only
pyb_df = pyb_df[pyb_df["game_type"] == "R"]

In [4]:
# Display the first 5 records of the dataframe
pyb_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
2618,CH,2024-05-10,90.1,-2.12,5.81,"Cano, Yennier",606466,666974,field_out,hit_into_play,...,1,4,2,1,1,1,2.95,1.22,-1.22,23.9
2670,CH,2024-05-10,90.4,-2.11,5.76,"Cano, Yennier",606466,666974,NaN,ball,...,1,4,2,1,1,1,2.77,1.41,-1.41,21.4
2731,CH,2024-05-10,91.8,-2.22,5.85,"Cano, Yennier",656976,666974,field_out,hit_into_play,...,1,0,2,1,1,3,2.53,1.25,-1.25,32.0
2829,CH,2024-05-10,90.4,-1.98,5.89,"Cano, Yennier",656976,666974,NaN,called_strike,...,1,0,2,1,1,3,2.51,1.41,-1.41,24.8
2952,CH,2024-05-10,90.3,-2.03,5.83,"Cano, Yennier",656976,666974,NaN,ball,...,1,0,2,1,1,3,3.0,1.56,-1.56,25.5


# Calculate observed movement angle(tilt angle) and spin efficiency

In [5]:
'''
function to calculate tilt angle(phi) and spin efficiency(spin_eff)
[caution] spin efficiency(spin_eff) is inappropriate for actual use due to large deviation of values from Baseball Savant.
'''
def break_tilt_cal(df):
    # Velocity and position calculation
    yR = 60 - df.release_extension
    tR = (-df.vy0 - (df.vy0**2 - 2 * df.ay * (50 - yR))**0.5) / df.ay
    vxR = df.vx0 + df.ax * tR
    vyR = df.vy0 + df.ay * tR
    vzR = df.vz0 + df.az * tR
    dv0 = df.release_speed - (vxR**2 + vyR**2 + vzR**2)**0.5 / 1.467

    # Flight time and movement distance calculation
    tf = (-vyR - (vyR**2 - 2 * df.ay * (yR - 17/12))**0.5) / df.ay
    x_mvt = df.plate_x - df.release_pos_x - (vxR / vyR) * (17/12 - yR)
    z_mvt = df.plate_z - df.release_pos_z - (vzR / vyR) * (17/12 - yR) + 0.5 * 32.174 * tf**2

    # Average velocity vector calculation
    vxbar = (2 * vxR + df.ax * tf) / 2
    vybar = (2 * vyR + df.ay * tf) / 2
    vzbar = (2 * vzR + df.az * tf) / 2
    vbar = (vxbar**2 + vybar**2 + vzbar**2)**0.5

    # Air resistance and coefficient calculation
    adrag = -(df.ax * vxbar + df.ay * vybar + (df.az + 32.174) * vzbar) / vbar
    Cd = adrag / (5.153E-03 * vbar**2)

    # Magnus force vector calculation
    amagx = df.ax + adrag * vxbar / vbar
    amagy = df.ay + adrag * vybar / vbar
    amagz = df.az + adrag * vzbar / vbar + 32.174
    amag = (amagx**2 + amagy**2 + amagz**2)**0.5

    # Ball movement and spin calculation
    Mx = 0.5 * amagx * tf**2 * 12
    Mz = 0.5 * amagz * tf**2 * 12
    Cl = amag / (5.153E-03 * vbar**2)
    S = 0.4 * Cl / (1 - 2.32 * Cl)
    spinT = 78.92 * S * vbar

    # Spin axis direction calculation
    spinTX = spinT * (vybar * amagz - vzbar * amagy) / (amag * vbar)
    spinTY = spinT * (vzbar * amagx - vxbar * amagz) / (amag * vbar)
    spinTZ = spinT * (vxbar * amagy - vybar * amagx) / (amag * vbar)
    spin_check = (spinTX**2 + spinTY**2 + spinTZ**2)**0.5 - spinT

    # Tilt angle (phi) calculation
    phi = (np.arctan2(amagz, amagx) * 180 / math.pi + 90)
    phi = phi + amagz.apply(lambda x: 360 if x < 0 else 0)
    phi = phi % 360

    # Spin efficiency calculation
    spin_eff = spinT / df.release_spin_rate

    # Add calculation results to the original dataframe
    df['phi'] = phi
    df['spin_eff'] = spin_eff

    return df

In [6]:
# Function Execution (add fields to dataframe)
pyb_df_tilt = break_tilt_cal(pyb_df)

# Display the first 5 records of the dataframe
pyb_df_tilt.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,phi,spin_eff
2618,CH,2024-05-10,90.1,-2.12,5.81,"Cano, Yennier",606466,666974,field_out,hit_into_play,...,2,1,1,1,2.95,1.22,-1.22,23.9,282.620024,0.74498
2670,CH,2024-05-10,90.4,-2.11,5.76,"Cano, Yennier",606466,666974,NaN,ball,...,2,1,1,1,2.77,1.41,-1.41,21.4,273.925305,0.932189
2731,CH,2024-05-10,91.8,-2.22,5.85,"Cano, Yennier",656976,666974,field_out,hit_into_play,...,2,1,1,3,2.53,1.25,-1.25,32.0,267.119172,0.758091
2829,CH,2024-05-10,90.4,-1.98,5.89,"Cano, Yennier",656976,666974,NaN,called_strike,...,2,1,1,3,2.51,1.41,-1.41,24.8,263.32791,0.884324
2952,CH,2024-05-10,90.3,-2.03,5.83,"Cano, Yennier",656976,666974,NaN,ball,...,2,1,1,3,3.0,1.56,-1.56,25.5,281.119599,1.19758
